In [2]:
// Define GCS paths
val bucketName = "movielens_dataset"
val ratingsPath = s"gs://$bucketName/rating.csv"

lastException = null
bucketName = movielens_dataset
ratingsPath = gs://movielens_dataset/rating.csv


gs://movielens_dataset/rating.csv

In [4]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("Movielens User Ratings Partitioning")
  .getOrCreate()

val ratingsDF = spark.read
  .format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(ratingsPath)

lastException = null
spark = org.apache.spark.sql.SparkSession@7e8a58a1
ratingsDF = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [8]:
// Get distinct user IDs and sample 30 users
val userSubset = ratingsDF.select("userId").distinct().limit(30)
val userSubsetList = userSubset.collect().map(_.getInt(0)) // Convert to Array[Int]

userSubset = [userId: int]
userSubsetList = Array(148, 463, 471, 496, 833, 1088, 1238, 1342, 1580, 1591, 1645, 1829, 1959, 2122, 2142, 2366, 2659, 2866, 3175, 3749, 3794, 3918, 3997, 4101, 4519, 4818, 4900, 4935, 5156, 5300)


Array(148, 463, 471, 496, 833, 1088, 1238, 1342, 1580, 1591, 1645, 1829, 1959, 2122, 2142, 2366, 2659, 2866, 3175, 3749, 3794, 3918, 3997, 4101, 4519, 4818, 4900, 4935, 5156, 5300)

In [13]:
import org.apache.spark.sql.functions._


In [14]:
val smallRatingsDF = ratingsDF.filter(col("userId").isin(userSubsetList: _*))

smallRatingsDF = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [15]:
println(s"Ratings: ${smallRatingsDF.count()}")

Ratings: 4421


In [16]:
val filteredDF = smallRatingsDF.filter("userId IS NOT NULL AND movieId IS NOT NULL AND rating IS NOT NULL")


filteredDF = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [17]:
smallRatingsDF.show(10)


+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|   148|      7|   4.0|2002-04-16 14:27:44|
|   148|     17|   5.0|2002-04-16 14:22:47|
|   148|     18|   1.0|2002-04-16 14:07:27|
|   148|     25|   2.0|2002-04-16 14:28:15|
|   148|     39|   4.0|2002-04-16 14:24:46|
|   148|     46|   4.0|2002-04-16 14:25:24|
|   148|     58|   4.0|2002-04-16 14:29:01|
|   148|     74|   3.0|2002-04-16 14:29:01|
|   148|     86|   2.0|2002-04-16 14:06:04|
|   148|    163|   1.0|2002-04-16 14:29:01|
+------+-------+------+-------------------+
only showing top 10 rows



In [18]:
val outputPath = s"hdfs:///user/caseStudy2"
// Partition by userId and write to HDFS
smallRatingsDF.write
  .option("header", "true")
  .partitionBy("userId")
  .csv(outputPath)

outputPath = hdfs:///user/caseStudy2


hdfs:///user/caseStudy2

In [21]:
val userFilePath = s"hdfs:///user/caseStudy2/userId=1088"
// Load the user's data into a DataFrame
val userDF = spark.read
  .option("header", "true")
  .csv(userFilePath)

// Show the data
println(s"Data for userId:1088")
userDF.show()

Data for userId:1088
+-------+------+--------------------+
|movieId|rating|           timestamp|
+-------+------+--------------------+
|      5|   3.0|1997-03-10T14:08:...|
|      9|   4.0|1997-03-10T14:10:...|
|     12|   3.0|1997-03-10T14:11:...|
|     26|   1.0|1997-03-10T14:13:...|
|     32|   3.0|1997-03-10T14:08:...|
|     61|   4.0|1997-03-10T14:12:...|
|     62|   3.0|1997-03-10T14:08:...|
|     63|   2.0|1997-03-10T14:14:...|
|     66|   3.0|1997-03-10T14:11:...|
|     74|   3.0|1997-03-10T14:11:...|
|     76|   4.0|1997-03-10T14:11:...|
|     78|   3.0|1997-03-10T14:17:...|
|     79|   3.0|1997-03-10T14:10:...|
|     86|   3.0|1997-03-10T14:12:...|
|     92|   2.0|1997-03-10T14:13:...|
|     95|   4.0|1997-03-10T14:08:...|
|    102|   1.0|1997-03-10T14:12:...|
|    140|   3.0|1997-03-10T14:10:...|
|    260|   3.0|1997-03-10T14:09:...|
|    376|   4.0|1997-03-10T14:09:...|
+-------+------+--------------------+
only showing top 20 rows



userFilePath = hdfs:///user/caseStudy2/userId=1088
userDF = [movieId: string, rating: string ... 1 more field]


[movieId: string, rating: string ... 1 more field]

In [22]:
spark.stop()